In [ ]:
import geopandas as gpd 
from shapely.geometry import LineString

In [ ]:
MHWL = gpd.read_file('nz-MHWL.shp')
transect_rates = gpd.read_file('AOI_rates.shp')

In [ ]:
extended_transects = gpd.GeoDataFrame(columns =['transect_id','intersect_point'], geometry='geometry')

In [ ]:
for index, transect in transect_rates.interrows():
    if transect['geometry'].intersects(MHWL.unary_union):
        extended_transects = extended_transects.append({'transect_id':transect['id'], 
                                                        'intersect_point': None, 
                                                        'geometry': transect['geometry']}, 
                                                        ignore_index=True)
    else:
        intersect_point = transect ['geometry'].intersection(MHWL.unary_union)
        extended_line = LineString([transect['geometry'].coords[0], intersect_point])
        extended_transects = extended_transects.append({'transect_id': transect['id'], 
                                                        'intersect_point': intersect_point, 
                                                        'geometry': extended_line}, 
                                                        ignore_index=True)

In [ ]:
extended_transects.to_file('extended_transects.geojson')